In [ ]:
import MicroQuote
import threading
import time

Str_Account = 'masterlink'
Str_Password = 'ML2856'
Str_Server_Login = '202.39.34.86:443'
Str_Server_Quote = '202.39.34.86:7777'
Str_Server_Recover = '202.39.34.86:443'
Str_Product_Info = '202.39.34.86:443'

#   交易商品代碼，如：TXFB9、TXO11000F8、2330。若為I060，則為指數代號，如：TXF。
#   若為價差商品用左斜線分隔，如：TXFD9/E9。可用符號 | 分隔取多商品，但限於同
#   一商品種類。（詳細命名規則請參考期交所網站）。若要取得股票指數，輸入TAIEX，
#   但無法與其他商品同使取報價(第二項參數為Stock)
Str_Product = ''
#   交易商品種類，共下列4 項：Future、Option、Stock、Warrant
Str_Product_Type = ''
#   報價資料種類，共下列4 項：I020、I030、I080、I060。證券與權值輸入空值
Str_Quote_Type = ''

Dic_Result_LogIn = {-3: '本程式已登入，請先執行登出', -1: '無法連線伺服器', 0: '資料庫異常',
                    1: '正常', 2: '密碼錯誤', 3: '無此帳號',
                    4: '無權限', 6: '聯絡客服人員', 7: '聯絡客服人員'}

#   -----------------------------------------------------------------------------   Class

Threads_test = []

class Thread_Order_RealTime(threading.Thread):
    def __init__(self, prod, prod_type, quote_type, thread_name=None):
        threading.Thread.__init__(self)
        self.prod = prod
        self.prod_type = prod_type
        self.quote_type = quote_type
        self.thread_stop = False
        self._stop_event = threading.Event()
        if thread_name is not None:
            self.name = thread_name

    def run(self):
        """
                data = MicroQuote.MicroQuote_Set(self.prod, self.prod_type, self.quote_type, False)

                for i in data.quote_on_notify():
                    if (self.thread_stop is True) or (i == -4):
                        break
                    print('----------------------------------------------------')
                    print("即時報價 {0}_{1}_{2} : {3}".format(self.prod, self.prod_type, self.quote_type, i))
                    time.sleep(0.01)"""

        def target_func():
            data = MicroQuote.MicroQuote_Set(self.prod, self.prod_type, self.quote_type, False)
            for i in data.quote_on_notify():
                if (self.thread_stop is True) or (i == -4):
                    break
                print('----------------------------------------------------')
                print("即時報價 {0}_{1}_{2} : {3}".format(self.prod, self.prod_type, self.quote_type, i))
                time.sleep(0.01)

        subthread = threading.Thread(target=target_func, args=())
        subthread.setDaemon(True)   # daemon的特性：父线程退出时子线程就自动退出
        subthread.start()

        while not self.thread_stop:
            subthread.join(1)

        #print('Thread stopped')

    def stop(self):
        self.thread_stop = True
        self._stop_event.set()

    def stopped(self):
        return self._stop_event.is_set()

#   -----------------------------------------------------------------------------   Function

def SmartAPI_Login():
    global Str_Account, Str_Password, Str_Server_Login, Str_Server_Quote, Str_Server_Recover, Str_Product_Info
    return MicroQuote.MicroQuote_Login(
        Str_Account, Str_Password, Str_Server_Login, Str_Server_Quote, Str_Server_Recover, Str_Product_Info)

def Set_Quote():
    global Str_Product, Str_Product_Type

    print('輸入訂閱商品 : ')
    Str_Product = input().upper()
    print('輸入訂閱商品類別  1:Future 2:Option 3:Stock 4:Warrant')
    Str_Product_Type = input()

    if Str_Product and Str_Product_Type:
        if Str_Product == 'TAIEX':
            Str_Product_Type = '3'

        s_order = {'0': Other, '1': Future, '2': Option, '3': Stock, '4': Warrant}
        s_order.get(Str_Product_Type, '0')(Str_Product)

def Get_Input_Enter():
    global Threads_test
    keystrk = input('Press Enter to Exit \n')
    # thread doesn't continue until key is pressed
    #print('You pressed: ', keystrk)
    if keystrk == '':
        for t in Threads_test:
            t.stop()
            print('Thread: {0} , Stop : {1}'.format(t.name, t.thread_stop))

def Future(prod):
    global Threads_test
    Threads_test = []
    Threads_test.append(Thread_Order_RealTime(prod, 'Future', 'I020', 'Thread_Future_I020'))
    Threads_test.append(Thread_Order_RealTime(prod, 'Future', 'I030', 'Thread_Future_I030'))
    Threads_test.append(Thread_Order_RealTime(prod, 'Future', 'I080', 'Thread_Future_I080'))
    threading.Thread(target=Get_Input_Enter).start()

    print('Future {0} Quote Start'.format(prod))
    for t in Threads_test:
        t.start()

    for t in Threads_test:
        t.join()
        print('{0} isAlive: {1}'.format(t.name, t.isAlive()))
    print('Future {0} Quote End'.format(prod))
    Set_Quote()

def Option(prod):
    global Threads_test
    Threads_test = []
    Threads_test.append(Thread_Order_RealTime(prod, 'Option', 'I020', 'Thread_Option_I020'))
    Threads_test.append(Thread_Order_RealTime(prod, 'Option', 'I030', 'Thread_Option_I030'))
    Threads_test.append(Thread_Order_RealTime(prod, 'Option', 'I080', 'Thread_Option_I080'))
    threading.Thread(target=Get_Input_Enter).start()

    print('Option {0} Quote Start'.format(prod))
    for t in Threads_test:
        t.start()

    for t in Threads_test:
        t.join()
        print('{0} isAlive: {1}'.format(t.name, t.isAlive()))
    print('Option {0} Quote End'.format(prod))
    Set_Quote()

def Stock(prod):
    global Threads_test
    Threads_test = []
    Threads_test.append(Thread_Order_RealTime(prod, 'Stock', '', 'Thread_Stock'))
    threading.Thread(target=Get_Input_Enter).start()

    print('Stock {0} Quote Start'.format(prod))
    for t in Threads_test:
        t.start()

    for t in Threads_test:
        t.join()
        print('{0} isAlive: {1}'.format(t.name, t.isAlive()))
    print('Stock {0} Quote End'.format(prod))
    Set_Quote()

def Warrant(prod):
    global Threads_test
    Threads_test = []
    Threads_test.append(Thread_Order_RealTime(prod, 'Warrant', '', 'Thread_Warrant'))
    threading.Thread(target=Get_Input_Enter).start()

    print('Warrant {0} Quote Start'.format(prod))
    for t in Threads_test:
        t.start()

    for t in Threads_test:
        t.join()
        print('{0} isAlive: {1}'.format(t.name, t.isAlive()))
    print('Warrant {0} Quote End'.format(prod))
    Set_Quote()

def Other(prod):
    print('Other {0}'.format(prod))

#   -----------------------------------------------------------------------------   Main

try:

    if __name__ == '__main__':

        result_login = SmartAPI_Login()
        print("登入: {0}".format(Dic_Result_LogIn.get(result_login, '')))

        Set_Quote()

        #   訂閱商品
        """
        data = MicroQuote.MicroQuote_Set(Str_Product, Str_Product_Type, Str_Quote_Type, False)
        for i in data.quote_on_notify():
            print("及時報價 {0}_{1}_{2} : {3}".format(Str_Product, Str_Product_Type, Str_Quote_Type, i))"""

        #   取得一筆當前報價。可用符號 | 分隔取多商品。
        """
        data_Last = data.get_last_data(Str_Product_Type, Str_Product, Str_Quote_Type)
        print("{0}_{1}_{2} : {3}".format(Str_Product, Str_Product_Type, Str_Quote_Type, data_Last))"""

        #   可輸入起始以及 結束時間進行資料回補
        """
        for i in data.get_history_data('09:00:00', '10:00:00'):
            print("歷史資料回補 {0}_{1}_{2} : {3}".format(Str_Product, Str_Product_Type, Str_Quote_Type, i))"""

        #   訂閱交易商品後，進行資料回補並取得逐筆報價。True為回補最後一筆、 False為全回補，預設為 False
        """
        for i in data.recover_on_notify(False):
            print("回補並報價 {0}_{1}_{2} : {3}".format(Str_Product, Str_Product_Type, Str_Quote_Type, i))"""

        #   訂閱交易商品後，可取得該日商品列表
        """
        list_prod = data.get_list()
        print("該日商品列表數量{0}: {1}".format(Str_Product_Type, len(list_prod)))
        print("該日商品列表{0}: {1}".format(Str_Product_Type, list_prod))"""

        #   取得該日商品基本資料。可用逗號(,)分隔取多商品。
        """
        info_prod = data.get_product(Str_Product_Type, Str_Product)
        print("商品基本資料{0}: {1}".format(Str_Product, info_prod))"""

        #   取得該日商品盤後靜態資料。可用逗號(,)分隔取多商品。
        """
        data_Aft = data.get_after_hour(Str_Product_Type, Str_Product)
        print("商品盤後靜態資料{0}: {1}".format(Str_Product, data_Aft))"""

        #   取得盤中高低價。可用逗號(,)分隔取多商品。
        """
        data_OHL = data.get_OHL(Str_Product_Type, Str_Product)
        print("商品盤中高低價{0}: {1}".format(Str_Product, data_OHL))"""

        #   回傳伺服器時間
        """
        for i in data.heart_beat_server_time():
            print("伺服器時間 : {0}".format(i))"""

        #   訂閱股票指數，可搭配迴圈取得逐筆報價
        """
        for i in data.queue_on_taiex():
            print("指數 {0}_{1}_{2} : {3}".format(Str_Product, Str_Product_Type, Str_Quote_Type, i))"""

        #   訂閱盤中高低價 ，可搭配迴圈取得逐筆報價
        """
        for i in data.quote_on_OHL():
            print("盤中高低價 {0}_{1}_{2} : {3}".format(Str_Product, Str_Product_Type, Str_Quote_Type, i))"""

        #   取得帳務權限狀態
        """
        val_User_Permission = data.user_permission
        print("取得帳務權限狀態: {0}".format(val_User_Permission))"""

except Exception as ex:
    print("Error: {0}".format(ex))


登入: 正常
輸入訂閱商品 : 
txfk9
輸入訂閱商品類別  1:Future 2:Option 3:Stock 4:Warrant
1
Future TXFK9 Quote Start
----------------------------------------------------
即時報價 TXFK9_Future_I080 : ['TXFK9', '09:45:14.005000', '11473.00', '6', '11472.00', '44', '11471.00', '68', '11470.00', '95', '11469.00', '88', '11474.00', '37', '11475.00', '48', '11476.00', '46', '11477.00', '67', '11478.00', '35', '1', '11472.00', '1', '11475.00', '2']
----------------------------------------------------
即時報價 TXFK9_Future_I020 : ['TXFK9', '09:45:14.005000', '09:45:13.880000', '11473.00', '1', '128', '28589', '14316', '14515']
----------------------------------------------------
----------------------------------------------------
即時報價 TXFK9_Future_I020 : ['TXFK9', '09:45:14.130000', '09:45:14.005000', '11473.00', '3', '128', '28592', '14317', '14516']
即時報價 TXFK9_Future_I080 : ['TXFK9', '09:45:14.130000', '11472.00', '44', '11471.00', '68', '11470.00', '95', '11469.00', '88', '11468.00', '90', '11474.00', '37', '11475.00'

----------------------------------------------------
即時報價 TXFK9_Future_I080 : ['TXFK9', '09:45:17.130000', '11473.00', '26', '11472.00', '54', '11471.00', '69', '11470.00', '92', '11469.00', '88', '11474.00', '21', '11475.00', '44', '11476.00', '48', '11477.00', '70', '11478.00', '36', '1', '11472.00', '2', '11475.00', '1']
----------------------------------------------------
即時報價 TXFK9_Future_I080 : ['TXFK9', '09:45:17.255000', '11473.00', '27', '11472.00', '54', '11471.00', '69', '11470.00', '92', '11469.00', '88', '11474.00', '21', '11475.00', '44', '11476.00', '48', '11477.00', '70', '11478.00', '36', '1', '11472.00', '2', '11475.00', '1']
----------------------------------------------------
即時報價 TXFK9_Future_I080 : ['TXFK9', '09:45:17.380000', '11473.00', '26', '11472.00', '54', '11471.00', '69', '11470.00', '92', '11469.00', '88', '11474.00', '21', '11475.00', '44', '11476.00', '48', '11477.00', '70', '11478.00', '36', '1', '11472.00', '2', '11476.00', '6']
----------------------

----------------------------------------------------
即時報價 TXFK9_Future_I080 : ['TXFK9', '09:45:19.880000', '11472.00', '40', '11471.00', '51', '11470.00', '67', '11469.00', '68', '11468.00', '81', '11474.00', '35', '11475.00', '49', '11476.00', '47', '11477.00', '70', '11478.00', '35', '1', '11471.00', '2', '11475.00', '2']
--------------------------------------------------------------------------------------------------------
即時報價 TXFK9_Future_I020 : ['TXFK9', '09:45:20.005000', '09:45:19.894000', '11472.00', '2', '128', '28620', '14337', '14532']

即時報價 TXFK9_Future_I080 : ['TXFK9', '09:45:20.005000', '11472.00', '38', '11471.00', '50', '11470.00', '67', '11469.00', '68', '11468.00', '81', '11474.00', '36', '11475.00', '49', '11476.00', '47', '11477.00', '70', '11478.00', '35', '1', '11471.00', '2', '11475.00', '2']
----------------------------------------------------
即時報價 TXFK9_Future_I080 : ['TXFK9', '09:45:20.130000', '11472.00', '38', '11471.00', '50', '11470.00', '67', '11469.00'

----------------------------------------------------
即時報價 TXFK9_Future_I080 : ['TXFK9', '09:45:24.005000', '11473.00', '14', '11472.00', '39', '11471.00', '48', '11470.00', '64', '11469.00', '69', '11474.00', '22', '11475.00', '51', '11476.00', '46', '11477.00', '66', '11478.00', '37', '1', '11472.00', '2', '11475.00', '2']
----------------------------------------------------
即時報價 TXFK9_Future_I080 : ['TXFK9', '09:45:24.130000', '11473.00', '18', '11472.00', '40', '11471.00', '48', '11470.00', '64', '11469.00', '69', '11474.00', '22', '11475.00', '50', '11476.00', '46', '11477.00', '66', '11478.00', '37', '1', '11472.00', '2', '11475.00', '1']
----------------------------------------------------
即時報價 TXFK9_Future_I080 : ['TXFK9', '09:45:24.255000', '11473.00', '16', '11472.00', '40', '11471.00', '48', '11470.00', '64', '11469.00', '69', '11474.00', '22', '11475.00', '51', '11476.00', '46', '11477.00', '66', '11478.00', '37', '1', '11472.00', '2', '11475.00', '1']
----------------------

----------------------------------------------------
即時報價 TXFK9_Future_I080 : ['TXFK9', '09:45:28.130000', '11473.00', '13', '11472.00', '47', '11471.00', '51', '11470.00', '66', '11469.00', '69', '11474.00', '27', '11475.00', '51', '11476.00', '45', '11477.00', '66', '11478.00', '38', '1', '11472.00', '1', '11475.00', '2']
----------------------------------------------------
即時報價 TXFK9_Future_I080 : ['TXFK9', '09:45:28.255000', '11473.00', '18', '11472.00', '47', '11471.00', '51', '11470.00', '66', '11469.00', '69', '11474.00', '27', '11475.00', '51', '11476.00', '45', '11477.00', '66', '11478.00', '38', '1', '11472.00', '1', '11475.00', '2']
----------------------------------------------------
即時報價 TXFK9_Future_I080 : ['TXFK9', '09:45:28.380000', '11473.00', '24', '11472.00', '49', '11471.00', '50', '11470.00', '65', '11469.00', '69', '11474.00', '21', '11475.00', '50', '11476.00', '45', '11477.00', '66', '11478.00', '38', '1', '11472.00', '2', '11475.00', '1']
----------------------

...

----------------------------------------------------
----------------------------------------------------即時報價 TXFK9_Future_I080 : ['TXFK9', '09:45:49.755000', '11470.00', '30', '11469.00', '49', '11468.00', '88', '11467.00', '64', '11466.00', '86', '11471.00', '23', '11472.00', '57', '11473.00', '49', '11474.00', '59', '11475.00', '76', '1', '11469.00', '5', '11471.00', '1']

即時報價 TXFK9_Future_I020 : ['TXFK9', '09:45:49.755000', '09:45:49.669000', '11471.00', '1', '128', '28795', '14459', '14595']
----------------------------------------------------
即時報價 TXFK9_Future_I080 : ['TXFK9', '09:45:49.880000', '11470.00', '30', '11469.00', '49', '11468.00', '88', '11467.00', '64', '11466.00', '86', '11471.00', '23', '11472.00', '57', '11473.00', '49', '11474.00', '59', '11475.00', '76', '1', '11469.00', '5', '11472.00', '3']
--------------------------------------------------------------------------------------------------------
即時報價 TXFK9_Future_I020 : ['TXFK9', '09:45:50.005000', '09:45:49.96

--------------------------------------------------------------------------------------------------------
即時報價 TXFK9_Future_I080 : ['TXFK9', '09:45:52.255000', '11469.00', '22', '11468.00', '86', '11467.00', '65', '11466.00', '89', '11465.00', '62', '11470.00', '24', '11471.00', '54', '11472.00', '61', '11473.00', '49', '11474.00', '78', '1', '11469.00', '1', '11471.00', '3']

即時報價 TXFK9_Future_I020 : ['TXFK9', '09:45:52.255000', '09:45:52.196000', '11469.00', '1', '128', '28828', '14488', '14607']
----------------------------------------------------
即時報價 TXFK9_Future_I080 : ['TXFK9', '09:45:52.380000', '11469.00', '8', '11468.00', '86', '11467.00', '65', '11466.00', '89', '11465.00', '62', '11470.00', '29', '11471.00', '54', '11472.00', '61', '11473.00', '49', '11474.00', '78', '1', '11469.00', '1', '11471.00', '2']
----------------------------------------------------
即時報價 TXFK9_Future_I020 : ['TXFK9', '09:45:52.380000', '09:45:52.260000', '11469.00', '10', '128', '28838', '14496', '14

----------------------------------------------------
即時報價 TXFK9_Future_I080 : ['TXFK9', '09:45:53.630000', '11468.00', '33', '11467.00', '64', '11466.00', '89', '11465.00', '65', '11464.00', '58', '11469.00', '19', '11470.00', '47', '11471.00', '55', '11472.00', '61', '11473.00', '71', '1', '11468.00', '5', '11470.00', '2']
----------------------------------------------------
即時報價 TXFK9_Future_I080 : ['TXFK9', '09:45:53.755000', '11468.00', '33', '11467.00', '64', '11466.00', '89', '11465.00', '65', '11464.00', '58', '11469.00', '19', '11470.00', '48', '11471.00', '55', '11472.00', '61', '11473.00', '71', '1', '11468.00', '5', '11470.00', '2']
--------------------------------------------------------------------------------------------------------
即時報價 TXFK9_Future_I020 : ['TXFK9', '09:45:53.880000', '09:45:53.773000', '11468.00', '7', '128', '28895', '14544', '14635']

即時報價 TXFK9_Future_I080 : ['TXFK9', '09:45:53.880000', '11468.00', '29', '11467.00', '64', '11466.00', '89', '11465.00'

----------------------------------------------------
即時報價 TXFK9_Future_I080 : ['TXFK9', '09:45:56.255000', '11469.00', '25', '11468.00', '52', '11467.00', '66', '11466.00', '89', '11465.00', '65', '11470.00', '17', '11471.00', '54', '11472.00', '60', '11473.00', '69', '11474.00', '51', '1', '11468.00', '1', '11470.00', '1']
----------------------------------------------------
即時報價 TXFK9_Future_I020 : ['TXFK9', '09:45:56.380000', '09:45:56.299000', '11469.00', '1', '128', '28919', '14559', '14650']
----------------------------------------------------
即時報價 TXFK9_Future_I080 : ['TXFK9', '09:45:56.380000', '11469.00', '23', '11468.00', '53', '11467.00', '66', '11466.00', '89', '11465.00', '65', '11470.00', '17', '11471.00', '54', '11472.00', '60', '11473.00', '69', '11474.00', '51', '1', '11467.00', '6', '11470.00', '1']
----------------------------------------------------
即時報價 TXFK9_Future_I020 : ['TXFK9', '09:45:56.380000', '09:45:56.372000', '11469.00', '1', '128', '28920', '14560', '14

----------------------------------------------------
即時報價 TXFK9_Future_I080 : ['TXFK9', '09:45:59.005000', '11470.00', '8', '11469.00', '48', '11468.00', '54', '11467.00', '65', '11466.00', '87', '11471.00', '19', '11472.00', '59', '11473.00', '71', '11474.00', '56', '11475.00', '58', '1', '11469.00', '1', '11471.00', '1']
----------------------------------------------------
即時報價 TXFK9_Future_I080 : ['TXFK9', '09:45:59.130000', '11470.00', '9', '11469.00', '47', '11468.00', '54', '11467.00', '65', '11466.00', '87', '11471.00', '19', '11472.00', '60', '11473.00', '71', '11474.00', '56', '11475.00', '58', '1', '11469.00', '1', '11471.00', '1']
----------------------------------------------------
即時報價 TXFK9_Future_I080 : ['TXFK9', '09:45:59.255000', '11470.00', '13', '11469.00', '48', '11468.00', '55', '11467.00', '63', '11466.00', '87', '11471.00', '19', '11472.00', '60', '11473.00', '71', '11474.00', '56', '11475.00', '58', '1', '11469.00', '1', '11471.00', '1']
------------------------